In [1]:
%tensorflow_version 2.x
import librosa
import IPython
import numpy as np
import tensorflow as tf

TensorFlow 2.x selected.


# Problem 1

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train_dirty_male.wav to train_dirty_male.wav
Saving train_clean_male.wav to train_clean_male.wav
Saving test_x_01.wav to test_x_01.wav
Saving test_x_02.wav to test_x_02.wav


In [0]:
# read train data
s, sr = librosa.load('train_clean_male.wav', sr=None)
S_train = librosa.stft(s, n_fft=1024, hop_length=512)
sn_train, sr = librosa.load('train_dirty_male.wav', sr=None)
X_train = librosa.stft(sn_train, n_fft=1024, hop_length=512)

# read test data
sn_test1, sr = librosa.load('test_x_01.wav', sr=None)
X_test1 = librosa.stft(sn_test1, n_fft=1024, hop_length=512)
sn_test2, sr = librosa.load('test_x_02.wav', sr=None)
X_test2 = librosa.stft(sn_test2, n_fft=1024, hop_length=512)

abs_S_train, abs_X_train, abs_X_test1, abs_X_test2 = np.abs(S_train).T, np.abs(X_train).T, np.abs(X_test1).T, np.abs(X_test2).T

In [8]:
abs_X_train.shape[2]

IndexError: ignored

In [27]:
# Build the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(input_shape=(513, 1), #input_shape=(features, Channels)
                          filters=5,
                          kernel_size=3,
                          strides=2,
                          activation='relu'))
print(model.input_shape)
print(model.output_shape)

# model.add(tf.keras.layers.Conv1D(filters=5,
#                                  kernel_size=3,
#                                  strides=2,
#                                  activation='relu'))
# print(model.output_shape)

# model.add(tf.keras.layers.Dropout(rate=0.5))
# print(model.output_shape)

# model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
# print(model.output_shape)

# model.add(tf.keras.layers.Flatten())
# print(model.output_shape)

# model.add(tf.keras.layers.Dense(units=1024,
#                                 activation='relu'))
# print(model.output_shape)

# model.add(tf.keras.layers.Dense(units=513,
#                                 activation='relu'))
# print(model.output_shape)

model.compile(loss='categorical_crossentropy', optimizer='adam')

(None, 513, 1)
(None, 256, 5)


In [0]:
tf.reset_default_graph()

# initialize tensors
x = tf.placeholder(tf.float64, shape=(513, None))
y = tf.placeholder(tf.float64, shape=(513, None))
lr = tf.constant(0.0001)
init = tf.initializers.he_normal()

def create_tensors(m, n, H):
  W = tf.Variable(init([m, n], dtype=tf.float64), dtype=tf.float64)
  b = tf.Variable(init([n, 1], dtype=tf.float64), dtype=tf.float64)
  Y = tf.nn.relu(tf.transpose(W)@H + b)
  return W, b, Y

# implement 2 fully-connected hidden layers
Wi, bi, Yi = create_tensors(513, 1024, x)
W1, b1, Y1 = create_tensors(1024, 1024, Yi)
Wo, bo, Yo = create_tensors(1024, 513, Y1)

# loss and optimizer
loss = tf.losses.mean_squared_error(y, Yo)
opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [5]:
tf.executing_eagerly()

True